In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
import pandas as pd
import numpy as np

In [ ]:
train=pd.read_csv("/kaggle/input/tabular-playground-series-jun-2021/train.csv")
train

In [ ]:
train.info()

In [ ]:
test=pd.read_csv("/kaggle/input/tabular-playground-series-jun-2021/test.csv")
test

In [ ]:
test.info()

In [ ]:
## Used Label Encoder to Encode the Target Column:
from sklearn.preprocessing import LabelEncoder
label_encoder = LabelEncoder()
train['target'] = label_encoder.fit_transform(train['target'])

In [ ]:
import scipy.stats as stats
from scipy.stats import chi2_contingency

imp_var=[]
h=dict()
u=dict()
unimp_var=[]

class ChiSquare:
    def __init__(self, dataframe):
        self.df = dataframe
        self.p = None #This is The P-Value
        self.chi2 = None #Here we go for Chi Test Statistic
        self.dof = None
        
        self.dfTabular = None
        self.dfExpected = None
        
    def _print_chisquare_result(self, colX, alpha):
        if self.p<alpha:
          h[colX]=self.p
        else:
          unimp_var.append(colX)
          u[colX]=self.p
            
    def TestIndependence(self,colX,colY, alpha=0.05):
        X = self.df[colX].astype(str)
        Y = self.df[colY].astype(str)
        
        self.dfObserved = pd.crosstab(Y,X) 
        chi2, p, dof, expected = stats.chi2_contingency(self.dfObserved.values)
        self.p = p
        self.chi2 = chi2
        self.dof = dof 
        
        self.dfExpected = pd.DataFrame(expected, columns=self.dfObserved.columns, index = self.dfObserved.index)
        
        self._print_chisquare_result(colX, alpha)

In [ ]:
cT = ChiSquare(train)
testColumns = train.drop(columns=['target']).columns

for var in testColumns:
    cT.TestIndependence(colX=var,colY="target" )

In [ ]:
imp=pd.DataFrame()
m = sorted(h.items(), key=lambda x: x[1], reverse=False)
for i in range(len(m)):
  imp_var.append(m[i][0])
if(len(imp_var)>len(unimp_var)):
  for i in range(len(imp_var)-len(unimp_var)):
    unimp_var.append("")
else:
  for i in range(len(unimp_var)-len(imp_var)):
    imp_var.append("")
imp['Important Variables']=imp_var
imp['Unimportant Variables']=unimp_var
imp

In [ ]:
imp_var.append('target')
train=train[imp_var]

In [ ]:
x=train.drop(columns=['target'])
y=train['target']

In [ ]:
!pip install catboost
from catboost import CatBoostClassifier
cat = CatBoostClassifier(l2_leaf_reg=10,n_estimators=500,learning_rate=0.05,depth=5)
cat.fit(x, y)
cat_pred1=cat.predict_proba(test)
cat_pred = pd.DataFrame(cat_pred1)
prediction_cat = pd.concat([test['id'],cat_pred],axis=1)

In [ ]:
prediction_cat

In [ ]:
prediction_cat.columns=['id','Class_1','Class_2','Class_3','Class_4','Class_5','Class_6','Class_7','Class_8','Class_9']
prediction_cat.to_csv("cat_Results.csv",index=False)